In [3]:
%matplotlib inline
%load_ext tensorboard

import lightning
import torch
from lightning.pytorch.callbacks import EarlyStopping, ModelCheckpoint
from lightning.pytorch.loggers import TensorBoardLogger



The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
checkpoint_callback = ModelCheckpoint(monitor="val_loss", dirpath="logs/RoofSense", save_top_k=1, save_last=True)
early_stopping_callback = EarlyStopping(monitor="val_loss", min_delta=0.00, patience=50)

batch_size_callback = lightning.pytorch.callbacks.BatchSizeFinder()

logger = TensorBoardLogger(save_dir="logs/RoofSense")

In [ ]:
from classification.task import TrainingTask
from lightning import Trainer
from torchgeo.trainers import SemanticSegmentationTask

from classification.datamodules import TrainingDataModule

datamodule = TrainingDataModule(  # Dataset Options

    root="../training/test",  # Data Module Options
    batch_size=64, patch_size=64, num_workers=10, )
task = TrainingTask(model="fcn",
                                backbone="resnet18",
                                weights=True,
                                in_channels=6,
                                num_classes=10,
                                #inverse class frequency in complete dataset
                                #todo:try inverse class frequenct in training dataset.
                                #     have the data module compute it on the fly and return the corresponding tensor.
                                #                                 class_weights=torch.tensor([
                                # 0,
                                # 12.71247757,
                                # 27.29529363,
                                # 10.81809178,
                                # 0,
                                # 12.60368287,
                                # 771.0117647,
                                # 10.61655597,
                                # 6.209882977,
                                #                                     0,
                                # # 
                                # #                                 
                                #                                 ]),
                                #                                 num_filters=192,
                                loss="jaccard",
                                ignore_index=9,
                                lr=1e-3,
                                #use 1e-4 or 1e-3
                                # for scheduler
                                )


In [5]:
%tensorboard --logdir "logs/RoofSense" --host localhost --port 6506







In [ ]:
trainer = Trainer(callbacks=[checkpoint_callback, early_stopping_callback,
                             # lightning.pytorch.callbacks.BatchSizeFinder(),
                             # lightning.pytorch.callbacks.LearningRateFinder(),
                             lightning.pytorch.callbacks.GradientAccumulationScheduler(scheduling={0: 5}),
                             lightning.pytorch.callbacks.LearningRateMonitor()],
                  log_every_n_steps=1,
                  logger=logger,
                  max_epochs=1000,
                  # fast_dev_run=True

                  )

trainer.fit(model=task, datamodule=datamodule)

In [ ]:
trainer.state

In [ ]:
trainer.test(model=task, datamodule=datamodule)

In [ ]:
model = task.model.to("cpu").eval()
model

In [ ]:
from tqdm import tqdm
import numpy as np

y_preds = []
y_trues = []
for batch in tqdm(datamodule.test_dataloader()):
    images = batch["image"].to("cpu")
    y_trues.append(batch["mask"].numpy())
    with torch.inference_mode():
        y_pred = model(images).argmax(dim=1).cpu().numpy()
    y_preds.append(y_pred)

y_preds = np.concatenate(y_preds)
y_trues = np.concatenate(y_trues)

In [ ]:
for batch in datamodule.val_dataloader():
    print(batch["bbox"])

In [ ]:


minibatch_id = 5
img = images[minibatch_id, :, :, :]
msk = batch["mask"].to("cpu")[minibatch_id, :, :, :].squeeze()
prd = model(images).argmax(dim=1).cpu()[minibatch_id, :, :]



In [ ]:
y_preds

In [ ]:
import matplotlib.pyplot as plt

datamodule.plot(datamodule.val_dataset[0])

In [ ]:
plt.imshow(y_preds.squeeze())

In [ ]:
from sklearn.metrics import precision_score, recall_score

precision = precision_score(y_trues, y_preds, average="micro")
recall = recall_score(y_trues, y_preds, average="micro")
f1 = 2 * (precision * recall) / (precision + recall)
print(precision, recall, f1)